In [1]:
import jax
import optax
import wandb
import numpy as np
from os import path
import equinox as eqx
import jax.numpy as jnp
from etils import epath
from pickle import loads
from jax import lax, config
import orbax.checkpoint as ocp
from jax.tree_util import tree_map
from itertools import islice, chain, repeat

In [2]:
MODEL_DIM = EMB_SIZE = 256
SEQ_LEN = 64
VOCAB = 5_000

In [3]:
N_HEADS = 8
N_ENCODER_LAYERS = N_DECODER_LAYERS = 3
INPUT_VOCAB = OUTPUT_VOCAB = VOCAB

In [4]:
BATCH_SIZE = 32 
EPOCHS = 5

In [5]:
config.update("jax_debug_nans", True)
config.update("jax_softmax_custom_jvp", True)

In [6]:
def pickle_iterator(path):
    """
    Streaming itertor for pickled files.
    
    Parameters
    ----------
    path: str
        Path to the pickle file
    
    Returns
    -------
    Iterator
        pickle object iterator
    """
    with open(path, "rb") as f:
        cur_lst = []
        for pair in f:
            if pair == b'\n' and cur_lst[-1][-2:] == b'.\n':
                cur_lst.append(pair)
                yield loads(b''.join(cur_lst))
                cur_lst = []
            else:
                cur_lst.append(pair)

In [7]:
def create_mask(arr):
    """
    Creates a mask array in the same shape as the given a padded array.

    Parameters
    ----------
    arr: jax.Array
        Padded Array

    Returns
    -------
    jax.Array
        A mask array.
    """
    return jnp.where(arr == 0, np.NINF, 0)

In [8]:
def pad_or_truncate(seq, size=32, pad=0):
    """
    Pads and truncates a list tokens a specific length.

    Parameters:
    seq: list
        A list of tokens.
    size: int, optional
        Max size of the sequence. (default is 32)
    pad: int, optional
        The ID to use for padding. (defualt is 0)
    
    Returns
    -------
    list
        Truncated or padded list of tokens.
    """
    return list(islice(chain(seq, repeat(pad)), size))

In [9]:
def seq2seq_batched_iterator(data, seq_len, batch_size=32):
    """
    A dataloader for sequence to sequence tasks.
    Loads data in batches.
    
    Parameters
    ----------
    data: Iterator
        Pairs of Sequences.
    seq_len: int
        Max length of the sequence.
    batch_size: int, optional
        Size of a batch. (default is 32)
    
    Returns
    -------
    Iterator[tuple]
        Batches of X, y and labels
    """
    while batch := tuple(islice(data, batch_size)):
        X = [x[0] for x in batch]
        y = [x[1] for x in batch]
        
        Xbt = [pad_or_truncate(seq, seq_len) for seq in X]
        y = [pad_or_truncate(seq, seq_len+1) for seq in y]
        ybt = [x[:-1] for x in y]
        labelbt = [x[1:] for x in y]
        
        yield Xbt, ybt, labelbt

In [10]:
data_loader = seq2seq_batched_iterator(pickle_iterator("./data_splits/en_fr_train_tokenized.pickle"), SEQ_LEN)

In [11]:
class Linear(eqx.Module):
    weights: jax.Array
    bias: jax.Array
    use_bias: bool = eqx.field(static=True)

    def __init__(self, key, nin, nout, use_bias=False):
        init = jax.nn.initializers.he_uniform()
        self.weights = init(key=key, shape=(nin, nout))
        if use_bias:
            self.bias = jnp.ones(nout)
        else: 
            self.bias = None
        self.use_bias = use_bias
        
    @eqx.filter_jit
    def __call__(self, x):
        x = x @ self.weights
        if self.use_bias:
            x = x + self.bias
        return x

In [12]:
class FFNN(eqx.Module):
    layers: list
    def __init__(self, key, nin, nout, nhidden, n_layers=2, use_bias=False):
        keys = jax.random.split(key, num=n_layers)
        layers = [
            Linear(keys[0], nin, nhidden, use_bias)
        ]
        for i in range(1, n_layers-1):
            layers.append(jax.nn.gelu)
            layers.append(Linear(keys[i], nhidden, nhidden, use_bias))
        if n_layers != 1:
            layers.append(Linear(keys[-1], nhidden, nout, use_bias))
        self.layers = layers

    @eqx.filter_jit
    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

In [13]:
class MultiHeadAttention(eqx.Module):
    wquery: jax.Array
    wkey: jax.Array
    wvalue: jax.Array
    weights: jax.Array
    n_heads: int = eqx.field(static=True)
    dim_k: int = eqx.field(static=True)

    def __init__(self, key, n_heads, dim):
        if (dim % n_heads) != 0:
            raise ValueError("Model dimensions must be a multiple of no. of heads")
        dim_k = dim // n_heads
        init = jax.nn.initializers.he_uniform()
        wkey, qkey, kkey, vkey = jax.random.split(key, num=4)
        self.weights = init(key=wkey, shape=(n_heads * dim_k, dim))
        self.wquery = init(key=qkey, shape=(dim, dim))
        self.wkey = init(key=kkey,shape=(dim, dim))
        self.wvalue = init(key=vkey, shape=(dim, dim))
        self.n_heads = n_heads
        self.dim_k = dim_k

    @eqx.filter_jit
    def __call__(self, query, key, value, mask):
        query, key, value = query @ self.wquery, key @ self.wkey, value @ self.wvalue
        query, key, value = [jnp.transpose(jnp.reshape(x, (-1, self.n_heads, self.dim_k)), (1, 0, 2)) 
                             for x in (query, key, value)]
        if mask.ndim == 1:
            mask = jnp.expand_dims(mask, axis=0)
        # Attention Calculation
        scaled_dot_prod = query @ jnp.transpose(key, (0, 2, 1)) / jnp.sqrt(query.shape[-1])
        scaled_dot_prod = mask + scaled_dot_prod
        attn = jax.nn.softmax(scaled_dot_prod) @ value
        return jnp.reshape(jnp.transpose(attn, (1, 0, 2)), (-1, self.n_heads * self.dim_k)) @ self.weights

In [14]:
class LayerNorm(eqx.Module):
    gamma: jax.Array
    bias: jax.Array
    eps: int = eqx.field(static=True)

    def __init__(self, size, eps=1e-6):
        self.gamma = jnp.ones(size)
        self.bias = jnp.ones(size)
        self.eps = 1e-6

    @eqx.filter_jit
    def __call__(self, x):
        mean = jnp.mean(x, -1, keepdims=True)
        std = jnp.std(x, -1, keepdims=True)
        return (self.gamma * (x - mean) / (std + self.eps)) + self.bias

In [15]:
class Encoder(eqx.Module):
    emb: jax.Array
    attn_layers: list
    ff_layers:list
    attn_norms: list
    ff_norms: list
    n_layers: int = eqx.field(static=True)
    pe: jax.Array = eqx.field(static=True)

    def __init__(self, key, n_layers, n_heads, dim, seq_len, vocab):
        keys = jax.random.split(key, num=n_layers*2+1)
        emb_key, attn_keys, ff_keys = keys[0], keys[1:n_layers+1], keys[n_layers+1:]
        self.emb = jax.random.normal(emb_key, (vocab, dim))
        # Self-Attention & Forward Layers
        self.attn_layers = [MultiHeadAttention(key, n_heads, dim) for key in attn_keys]
        self.ff_layers = [FFNN(key, dim, dim, dim*2) for key in ff_keys]
        # Layer Norms
        self.attn_norms = [LayerNorm(dim) for _ in range(n_layers)]
        self.ff_norms = [LayerNorm(dim) for _ in range(n_layers)]
        # Positional Encodings
        pos = jnp.arange(seq_len)[:, jnp.newaxis]
        div_term = 10_000 ** (2 * jnp.arange(0, dim, 2) / dim)
        pe = jnp.empty((seq_len, dim))
        pe = pe.at[:, 0::2].set(jnp.sin(pos / div_term))
        pe = pe.at[:, 1::2].set(jnp.cos(pos / div_term))
        self.pe = pe
        # Static Arguments
        self.n_layers = n_layers

    @eqx.filter_jit
    def __call__(self, x, mask):
        x = self.emb[x[...]]
        x = x + self.pe
        for i in range(self.n_layers):
            x = self.attn_norms[i](self.attn_layers[i](x, x, x, mask) + x)
            x = self.ff_norms[i](self.ff_layers[i](x) + x)
        return x

In [16]:
class Decoder(eqx.Module):
    emb: jax.Array
    mask: jax.Array = eqx.field(static=True)
    masked_attn_layers: list
    attn_layers: list
    ff_layers:list
    masked_attn_norms: list
    attn_norms: list
    ff_norms: list
    n_layers: int = eqx.field(static=True)
    pe: jax.Array = eqx.field(static=True)

    def __init__(self, key, n_layers, n_heads, dim, seq_len, vocab):
        keys = jax.random.split(key, num=n_layers*3+1)
        emb_key, attn_keys, ff_keys, masked_attn_keys = (
            keys[0], keys[1:n_layers+1], keys[n_layers+1:n_layers*2+1], keys[n_layers*2+1:]
        )
        self.emb = jax.random.normal(emb_key, (vocab, dim))
        self.mask = jnp.where(jnp.triu(jnp.ones((seq_len, seq_len)), 1) == 1, np.NINF, 0)
        # Masked-Attention, Self-Attention & Forward Layers
        self.masked_attn_layers = [MultiHeadAttention(key, n_heads, dim) for key in masked_attn_keys]
        self.attn_layers = [MultiHeadAttention(key, n_heads, dim) for key in attn_keys]
        self.ff_layers = [FFNN(key, dim, dim, dim*2) for key in ff_keys]
        # Layer Norms
        self.masked_attn_norms = [LayerNorm(dim) for _ in range(n_layers)]
        self.attn_norms = [LayerNorm(dim) for _ in range(n_layers)]
        self.ff_norms = [LayerNorm(dim) for _ in range(n_layers)]
        # Positional Encodings
        pos = jnp.arange(seq_len)[:, jnp.newaxis]
        div_term = 10_000 ** (2 * jnp.arange(0, dim, 2) / dim)
        pe = jnp.empty((seq_len, dim))
        pe = pe.at[:, 0::2].set(jnp.sin(pos / div_term))
        pe = pe.at[:, 1::2].set(jnp.cos(pos / div_term))
        self.pe = pe
        # Static Arguments
        self.n_layers = n_layers

    @eqx.filter_jit
    def __call__(self, x, m, x_mask, m_mask):
        x = self.emb[x[...]]
        x = x + self.pe
        x_mask = self.mask + x_mask
        for i in range(self.n_layers):
            x = self.masked_attn_norms[i](self.masked_attn_layers[i](x, x, x, x_mask) + x)
            x = self.attn_norms[i](self.attn_layers[i](x, m, m, m_mask) + x)
            x = self.ff_norms[i](self.ff_layers[i](x) + x)
        return x

In [17]:
class EncoderDecoder(eqx.Module):
    encoder: eqx.Module
    decoder: eqx.Module

    def __init__(self, key, dim, enc_heads, enc_layers, dec_heads, dec_layers, in_vocab, out_vocab, seq_len):
        enc_key, dec_key = jax.random.split(key, num=2)
        self.encoder = Encoder(enc_key, enc_layers, enc_heads, dim, seq_len, in_vocab)
        self.decoder = Decoder(dec_key, dec_layers, dec_heads, dim, seq_len, out_vocab)

    @eqx.filter_jit
    def __call__(self, X, y, X_mask, y_mask):
        m = self.encoder(X, X_mask)
        h = self.decoder(y, m, y_mask, X_mask)
        return h

In [18]:
class Transformer(eqx.Module):
    enc_dec: eqx.Module
    linear: eqx.Module

    def __init__(self, key, dim, enc_heads, enc_layers, dec_heads, dec_layers, in_vocab, out_vocab, seq_len):
        encdec_key, linear_key = jax.random.split(key)
        self.enc_dec = EncoderDecoder(encdec_key, dim,
                                      enc_heads, enc_layers, 
                                      dec_heads, dec_layers, 
                                      in_vocab, out_vocab, seq_len)
        self.linear = Linear(linear_key, dim, out_vocab)

    @eqx.filter_jit
    def __call__(self, X, y, X_mask, y_mask):
        logits = self.enc_dec(X, y, X_mask, y_mask)
        return jax.nn.softmax(self.linear(logits)) + 1e-9

In [19]:
key = jax.random.PRNGKey(0)
model = Transformer(key, MODEL_DIM, 
                    N_ENCODER_HEADS, N_ENCODER_LAYERS, 
                    N_DECODER_HEADS, N_DECODER_LAYERS, 
                    INPUT_VOCAB, OUTPUT_VOCAB, SEQ_LEN)

In [20]:
scheduler = optax.warmup_cosine_decay_schedule(0.01, 1, 200, 2000, 0.5)
optimizer = optax.adam(learning_rate=scheduler)

In [21]:
def predict(model, X, y, X_mask, y_mask):
    return model(X, y, X_mask, y_mask)

In [22]:
def loss(model, X, y, X_mask, y_mask, labels):
    y_pred = jnp.log(predict(model, X, y, X_mask, y_mask))
    y_pred = jnp.where(labels==0, 0, jnp.take(y_pred, labels, axis=-1))
    count = jnp.count_nonzero(y_pred)
    return -jnp.sum(y_pred)/count

In [23]:
def optim(model, optimizer, loss_fn, vectorize=True, in_axes=None, out_axes=None):
    opt_state = optimizer.init(model)
    grad = jax.value_and_grad(loss_fn)
    if vectorize:
        gradient = jax.vmap(grad, in_axes=in_axes, out_axes=out_axes)
    
    @eqx.filter_jit
    def step(model, opt_state, X, y, X_mask, y_mask, labels):
        loss_value, grads = gradient(model, X, y, X_mask, y_mask, labels)
        if vectorize:
            loss_value = jnp.mean(loss_value)
            grads = tree_map(lambda x: jnp.mean(x, axis=0), grads)
        updates, opt_state = optimizer.update(grads, opt_state, model)
        model = optax.apply_updates(model, updates)
        return model, opt_state, loss_value

    return opt_state, step

In [24]:
opt_state, step = optim(model, optimizer, loss, in_axes=(None, 0, 0, 0, 0, 0), out_axes=0)

In [25]:
model_path = "./checkpoint/nmt-transformer/model.eqx"
opt_state_path = "./checkpoint/nmt-transformer/opt_state.eqx"

In [26]:
if path.isfile(model_path) and path.isfile(opt_state_path):
    model = eqx.tree_deserialise_leaves(model_path, model)
    opt_state = eqx.tree_deserialise_leaves(opt_state_path, opt_state)

In [ ]:
PROJECT = "nmt-transformer"

run = wandb.init(
    "project": PROJECT,
    "notes": "Initial run"
)

In [ ]:
for e in range(0, EPOCHS):
    total_loss = 0
    num_batches = 0
    for i, (Xbt, ybt, labelbt) in enumerate(data_loader):
        if e*num_batches >= mngr.latest_step():
            Xbt, ybt, labelbt = [jnp.array(x) for x in (Xbt, ybt, labelbt)]
            Xmask, ymask = [create_mask(x) for x in (Xbt, ybt)]

            model, opt_state, batch_loss = step(model, opt_state, Xbt, ybt, Xmask, ymask, labelbt)
            total_loss += batch_loss
            num_batches += 1

            if num_batches % 25 == 0:
                eqx.tree_serialise_leaves(model_path, model)
                eqx.tree_deserialise_leaves(opt_state_path, opt_state)
                print(f"Batches trained: {num_batches} | Avg. Batch loss: {total_loss/num_batches}")

    epoch_loss = total_loss / num_batches
    print(f"Epoch {e} | loss: {epoch_loss}")